# Reading JSON File With Haskell

Author: Kat Chuang [@katychuang](http://twitter.com/katychuang) on Twitter

## Objective

The goal of this exercise is to parse JSON from the Facebook Graph Api using the Aeson library. 

I collected data from my most recent posts and previously saved json output from a [python version of this code](http://nbviewer.jupyter.org/github/katychuang/ipython-notebooks/blob/master/facebook_posting_activity.ipynb) and saved it into a `file.json` 

---

First import the modules we'll need for this exercise.

In [1]:
{-# LANGUAGE DeriveGeneric #-}

import Data.Aeson
import Data.Aeson.Types
import Data.Time.Clock (UTCTime, diffUTCTime, utctDay)
import Data.Time.Calendar
import qualified Data.ByteString.Lazy as B
import Data.Maybe (fromJust)
import Options.Generic

data Post = Post
  { created_time :: UTCTime
  , id           :: String
  , message      :: Maybe String
  , story        :: Maybe String
  } deriving (Show, Generic)

instance FromJSON Post
instance ToJSON Post

input <- B.readFile "_+posts2017.json"

We now have the `input` with the data read in from the JSON file. Now we can go ahead to parse the values.

Here's one way, using `eitherDecode`, which captures the error message.

In [2]:
let x = eitherDecode input :: Either String [Post]
recent <- case x of 
  Left err -> putStrLn err
  Right value -> mapM_ print $ take 1 [ created_time x | x <- value ]

{- Types
putStrLn :: String -> IO ()
print :: Show a => a -> IO ()
mapM_ :: (Monad m, Foldable t) => (a -> m b) -> t a -> m ()
Data.Text.take :: Int -> Text -> Text
-}

2017-03-19 22:23:36 UTC

We can be more concise with the code - assuming the JSON formatting is predictable - instead use `decode` while assigning the data to a variable. It auto parses the list of JSON dictionaries into a list of Haskell data record types. To show you how it works, we can print out one field from the first item in the list. 

In [3]:
let (Just y) = decode input :: Maybe [Post]

-- show most recent timestamp
print $ created_time $ head y

2017-03-19 22:23:36 UTC

Not that we have `y`, we can easily work with displaying the information in a number of ways

In [4]:
-- # Example showing listing and filters

-- show all the timestamps
print $ show (length y) ++ " items in the file. Showing them all with map "
print (map created_time y) --same as: print $ map (\post -> created_time post) y

print $ show (length y) ++ " items in the file. Showing them with utctDay and list comprehension "
print [ utctDay (created_time x) | x <- y ]

-- Practice filtering
--print [ created_time x | x <- y, (>=) (utctDay $ created_time x) (fromGregorian 2017 3 1) ]
let m = [ created_time x | x <- y, (>=) (utctDay $ created_time x) (fromGregorian 2017 3 1) ]
print $ show (length m) ++  " items posted in March" 
print m

"46 items in the file. Showing them all with map "

[2017-03-19 22:23:36 UTC,2017-03-19 21:45:38 UTC,2017-03-19 21:24:07 UTC,2017-03-19 17:26:43 UTC,2017-03-17 19:57:17 UTC,2017-03-16 05:57:48 UTC,2017-03-15 19:06:26 UTC,2017-03-12 19:49:29 UTC,2017-03-12 05:35:42 UTC,2017-03-11 03:48:44 UTC,2017-03-11 02:05:14 UTC,2017-03-10 19:10:38 UTC,2017-03-10 06:24:15 UTC,2017-03-10 06:09:01 UTC,2017-03-09 20:45:35 UTC,2017-03-09 20:19:24 UTC,2017-03-05 14:02:33 UTC,2017-03-05 12:31:59 UTC,2017-03-04 20:30:23 UTC,2017-03-02 09:45:44 UTC,2017-03-01 05:24:02 UTC,2017-02-28 20:22:52 UTC,2017-02-22 03:01:11 UTC,2017-02-17 15:18:44 UTC,2017-02-11 17:00:00 UTC,2017-02-10 02:00:00 UTC,2017-02-09 20:32:37 UTC,2017-02-08 22:55:17 UTC,2017-02-05 22:47:10 UTC,2017-02-05 15:59:21 UTC,2017-02-04 12:43:57 UTC,2017-02-02 14:15:57 UTC,2017-02-02 13:49:41 UTC,2017-01-30 05:57:13 UTC,2017-01-30 05:40:25 UTC,2017-01-29 22:48:25 UTC,2017-01-27 17:00:00 UTC,2017-01-25 20:41:15 UTC,2017-01-25 19:16:11 UTC,2017-01-25 03:00:00 UTC,2017-01-24 22:47:41 UTC,2017-01-18 02:2

"46 items in the file. Showing them with utctDay and list comprehension "

[2017-03-19,2017-03-19,2017-03-19,2017-03-19,2017-03-17,2017-03-16,2017-03-15,2017-03-12,2017-03-12,2017-03-11,2017-03-11,2017-03-10,2017-03-10,2017-03-10,2017-03-09,2017-03-09,2017-03-05,2017-03-05,2017-03-04,2017-03-02,2017-03-01,2017-02-28,2017-02-22,2017-02-17,2017-02-11,2017-02-10,2017-02-09,2017-02-08,2017-02-05,2017-02-05,2017-02-04,2017-02-02,2017-02-02,2017-01-30,2017-01-30,2017-01-29,2017-01-27,2017-01-25,2017-01-25,2017-01-25,2017-01-24,2017-01-18,2017-01-17,2017-01-15,2017-01-07,2017-01-05]

"21 items posted in March"

[2017-03-19 22:23:36 UTC,2017-03-19 21:45:38 UTC,2017-03-19 21:24:07 UTC,2017-03-19 17:26:43 UTC,2017-03-17 19:57:17 UTC,2017-03-16 05:57:48 UTC,2017-03-15 19:06:26 UTC,2017-03-12 19:49:29 UTC,2017-03-12 05:35:42 UTC,2017-03-11 03:48:44 UTC,2017-03-11 02:05:14 UTC,2017-03-10 19:10:38 UTC,2017-03-10 06:24:15 UTC,2017-03-10 06:09:01 UTC,2017-03-09 20:45:35 UTC,2017-03-09 20:19:24 UTC,2017-03-05 14:02:33 UTC,2017-03-05 12:31:59 UTC,2017-03-04 20:30:23 UTC,2017-03-02 09:45:44 UTC,2017-03-01 05:24:02 UTC]

Having the time stamps, now we can try to replicate the process of creating nested lists data structure.

We have here a list, and can take the week information as well as the day information. The day of week information is the x of an (x,y) matrix; the week information is the y of an (x,y) matrix. 



In [5]:
import Data.Time.Calendar.WeekDate
import Data.Time.Format

-- print week numbers of each
print [ formatTime defaultTimeLocale "%U" x | x <- m]

-- print the day of the week
print [ formatTime defaultTimeLocale "%a" x | x <- m]


["12","12","12","12","11","11","11","11","11","10","10","10","10","10","10","10","10","10","09","09","09"]

["Sun","Sun","Sun","Sun","Fri","Thu","Wed","Sun","Sun","Sat","Sat","Fri","Fri","Fri","Thu","Thu","Sun","Sun","Sat","Thu","Wed"]

Next step is to try making this more seamless into creating a nested list by incorporating the index of lists

In [6]:
march = [ created_time x | x <- y, (>=) (utctDay $ created_time x) (fromGregorian 2017 3 1) ]
getXY u = [(formatTime defaultTimeLocale "%U" u), formatTime defaultTimeLocale "%u" u]
print [ getXY x | x <- march]

getX r = i - 9 where i = read $ (formatTime defaultTimeLocale "%U" r) :: Int
getY r = read $ (formatTime defaultTimeLocale "%U" r) :: Int

getXY' u = [getX u, getY u]
coords = [ getXY x | x <- march]
print coords


[["12","7"],["12","7"],["12","7"],["12","7"],["11","5"],["11","4"],["11","3"],["11","7"],["11","7"],["10","6"],["10","6"],["10","5"],["10","5"],["10","5"],["10","4"],["10","4"],["10","7"],["10","7"],["09","6"],["09","4"],["09","3"]]

[["12","7"],["12","7"],["12","7"],["12","7"],["11","5"],["11","4"],["11","3"],["11","7"],["11","7"],["10","6"],["10","6"],["10","5"],["10","5"],["10","5"],["10","4"],["10","4"],["10","7"],["10","7"],["09","6"],["09","4"],["09","3"]]

In [7]:
getWeek m j = m !! j -- returns a list
getDay m w d = m !! w !! d

week = take 7 (repeat 1)
month = take 5 (repeat week)

-- Takes a list and an index, edits element at index
mark w d = frt ++ (e: tail end)
    where 
        s = d - 1
        splice = splitAt s w
        frt = fst splice
        end = snd splice
        norm = 1 / 21
        e = (head end) - norm
        
-- show example of using mark
mark week 4




[1.0,1.0,1.0,0.9523809523809523,1.0,1.0,1.0]

In [8]:
import Plots.Axis  (Axis, r2Axis)
import Plots.Axis.ColourBar (colourBar)
import Plots.Axis.Render (renderAxis)
import Plots.Axis.Scale (axisExtend, noExtend)
import Plots.Style (axisColourMap, magma, greys)
import Plots.Types (display)
import Plots.Types.HeatMap

import Control.Lens ((&~), (.=))
import Diagrams.Backend.Cairo (B, Cairo)
import Diagrams.TwoD.Types (V2)
import Diagrams.Core.Types (QDiagram)
import IHaskell.Display.Diagrams
import IHaskell.Display.Juicypixels hiding (display)



In [ ]:
heatMapAxis' :: Axis Cairo V2 Double
heatMapAxis' = r2Axis &~ do
  display colourBar
  axisExtend Control.Lens..= noExtend
  axisColourMap Control.Lens..= greys
  let result = [[1..3], [3..1],[1..3]]
  heatMap' result

heatMapExample' = renderAxis heatMapAxis'
diagram heatMapExample'